In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
def formatFeature(features, data):
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
    features += ['Store', 'CompetitionDistance', 'Promo', 'Promo2', 'SchoolHoliday', \
                 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek',\
                 'Promo2SinceYear']

    features.extend(['StoreType', 'Assortment', 'StateHoliday'])
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    features.extend(['DayOfWeek', 'Month', 'Day', 'Year', 'WeekOfYear'])
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    features.append('CompetitionOpen')
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    features.append('PromoOpen')
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    
    features.extend(['IsPromoMonth', 'IsPromoNextMonth'])
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data['NextMonth'] = data.Month + 1
    data['nextMonthStr'] = data.NextMonth.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    data['IsPromoNextMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval),\
                         'IsPromoMonth'] = 1
                data.loc[(data.nextMonthStr == month) & (data.PromoInterval == interval),\
                         'IsPromoNextMonth'] = 1
    

In [37]:
googleTrends = pd.read_csv("../data/Rossmann_DE.csv")
googleTrends

,Week,rossmann
0,2004-01-04 - 2004-01-10,8
1,2004-01-11 - 2004-01-17,10
2,2004-01-18 - 2004-01-24,9
3,2004-01-25 - 2004-01-31,11
4,2004-02-01 - 2004-02-07,9
5,2004-02-08 - 2004-02-14,8
6,2004-02-15 - 2004-02-21,10
7,2004-02-22 - 2004-02-28,12
8,2004-02-29 - 2004-03-06,10
9,2004-03-07 - 2004-03-13,9


In [ ]:
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("../data/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("../data/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("../data/store.csv")

train.fillna(1, inplace=True)
test.fillna(1, inplace=True)

# Consider only open stores for training. Closed stores wont count into the score.
train = train[train["Open"] != 0]
# Use only Sales bigger then zero. Simplifies calculation of rmspe
train = train[train["Sales"] > 0]

In [22]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

features = []
formatFeature(features, train)
formatFeature([], test)

In [26]:
train[features].sample(3)

,Store,CompetitionDistance,Promo,Promo2,SchoolHoliday,StoreType,Assortment,StateHoliday,DayOfWeek,Month,Day,Year,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth,IsPromoNextMonth
742937,984,440,0,1,0,3,1,0,3,9,11,2014,37,24177,21.00,0,1
127891,171,2640,1,0,1,1,1,0,1,8,5,2014,32,24176,24176.00,0,0
12594,17,50,1,1,1,1,1,0,2,10,8,2014,41,106,51.75,1,0


In [29]:
X = train[features].values
y = np.log1p(train.Sales)
test = test[features].values

In [34]:
np.save(open('../data/X_pickle','wb'), X)
np.save(open('../data/y_pickle','wb'), y)
np.save(open('../data/test_pickle','wb'), test)
np.save(open('../data/features_pickle','wb'), features)